In [1]:
import pandas as pd
import warnings
import re
warnings.filterwarnings("ignore")
#%env PYDEVD_DISABLE_FILE_VALIDATION=1
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [2]:
dataset=pd.read_csv('loan_approval_dataset.csv')
dataset

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected
4267,4268,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved


# Data Cleaning
Checking for special characters

In [3]:
dataset = pd.DataFrame(dataset)

# Define a regular expression for special characters
special_char_pattern = r'[^a-zA-Z0-9.\s]'

# Iterate over each column to check for special characters
for column in dataset.columns:
    print(f"Checking column: {column}")
    
    # Convert the column to string to handle non-string values
    contains_special_chars = dataset[column].astype(str).str.contains(special_char_pattern, regex=True)
    
    # Print the rows where special characters are found
    if contains_special_chars.any():
        print(f"Rows with special characters in {column}:")
        print(dataset[contains_special_chars])
    else:
        print(f"No special characters found in {column}.")
    print()  # For better readability between columns

Checking column: loan_id
No special characters found in loan_id.

Checking column: no_of_dependents
No special characters found in no_of_dependents.

Checking column: education
No special characters found in education.

Checking column: self_employed
No special characters found in self_employed.

Checking column: income_annum
No special characters found in income_annum.

Checking column: loan_amount
No special characters found in loan_amount.

Checking column: loan_term
No special characters found in loan_term.

Checking column: cibil_score
No special characters found in cibil_score.

Checking column: residential_assets_value
Rows with special characters in residential_assets_value:
      loan_id  no_of_dependents      education self_employed  income_annum  \
59         60                 4   Not Graduate           Yes       5500000   
196       197                 4   Not Graduate           Yes        400000   
559       560                 2       Graduate           Yes        200000

In [4]:
#Removing special characters from residential assets value
dataset['residential_assets_value'].value_counts()

residential_assets_value
400000      66
500000      63
100000      60
1000000     57
600000      56
            ..
27500000     1
25000000     1
23100000     1
27400000     1
27300000     1
Name: count, Length: 278, dtype: int64

In [5]:
# Assuming your DataFrame is called `data` and the column is 'residential_assets_value'
dataset['residential_assets_value'] = dataset['residential_assets_value'].abs().astype(int)

In [6]:
dataset = pd.DataFrame(dataset)

# Define a regular expression for special characters
special_char_pattern = r'[^a-zA-Z0-9.\s]'

# Iterate over each column to check for special characters
for column in dataset.columns:
    print(f"Checking column: {column}")
    
    # Convert the column to string to handle non-string values
    contains_special_chars = dataset[column].astype(str).str.contains(special_char_pattern, regex=True)
    
    # Print the rows where special characters are found
    if contains_special_chars.any():
        print(f"Rows with special characters in {column}:")
        print(dataset[contains_special_chars])
    else:
        print(f"No special characters found in {column}.")
    print()  # For better readability between columns

Checking column: loan_id
No special characters found in loan_id.

Checking column: no_of_dependents
No special characters found in no_of_dependents.

Checking column: education
No special characters found in education.

Checking column: self_employed
No special characters found in self_employed.

Checking column: income_annum
No special characters found in income_annum.

Checking column: loan_amount
No special characters found in loan_amount.

Checking column: loan_term
No special characters found in loan_term.

Checking column: cibil_score
No special characters found in cibil_score.

Checking column: residential_assets_value
No special characters found in residential_assets_value.

Checking column: commercial_assets_value
No special characters found in commercial_assets_value.

Checking column: luxury_assets_value
No special characters found in luxury_assets_value.

Checking column: bank_asset_value
No special characters found in bank_asset_value.

Checking column: loan_status
No spec

# Checking for Null values

In [7]:
dataset.isna().sum()

loan_id                     0
no_of_dependents            0
education                   0
self_employed               0
income_annum                0
loan_amount                 0
loan_term                   0
cibil_score                 0
residential_assets_value    0
commercial_assets_value     0
luxury_assets_value         0
bank_asset_value            0
loan_status                 0
dtype: int64

# Checking for Zeros

In [8]:
dataset = pd.DataFrame(dataset)

# Iterate over each column to check for the value 0
for column in dataset.columns:
    print(f"Checking column: {column}")
    
    # Check if the column contains any zeros
    contains_zero = dataset[column] == 0
    
    # Print the rows where the value is zero
    if contains_zero.any():
        print(f"Rows with zero in {column}:")
        #print(df[contains_zero])
    else:
        print(f"No zero values found in {column}.")
    print()  # For better readability between columns

Checking column: loan_id
No zero values found in loan_id.

Checking column: no_of_dependents
Rows with zero in no_of_dependents:

Checking column: education
No zero values found in education.

Checking column: self_employed
No zero values found in self_employed.

Checking column: income_annum
No zero values found in income_annum.

Checking column: loan_amount
No zero values found in loan_amount.

Checking column: loan_term
No zero values found in loan_term.

Checking column: cibil_score
No zero values found in cibil_score.

Checking column: residential_assets_value
Rows with zero in residential_assets_value:

Checking column: commercial_assets_value
Rows with zero in commercial_assets_value:

Checking column: luxury_assets_value
No zero values found in luxury_assets_value.

Checking column: bank_asset_value
Rows with zero in bank_asset_value:

Checking column: loan_status
No zero values found in loan_status.



# Creating new columns

In [9]:
dataset['debt_to_income_ratio'] =dataset['loan_amount'] / dataset['income_annum']
dataset['total_assets_value'] = (dataset['residential_assets_value'] + dataset['commercial_assets_value'] +
                            dataset['luxury_assets_value'] + dataset['bank_asset_value'])
dataset['loan_to_asset_ratio'] = dataset['loan_amount'] / dataset['total_assets_value']
dataset

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved,3.114583,50700000,0.589744
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected,2.975610,17000000,0.717647
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected,3.263736,57700000,0.514731
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected,3.743902,52700000,0.582543
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected,2.469388,55000000,0.440000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected,2.300000,7400000,0.310811
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved,3.424242,20000000,0.565000
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected,3.676923,39000000,0.612821
4267,4268,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved,3.121951,28800000,0.444444


In [10]:
numerical_cols = dataset.select_dtypes(include=['number']).columns.to_list()
categorical_cols = dataset.select_dtypes(include=['object']).columns.to_list()

print("Numerical columns: ", numerical_cols)
print("Categorical columns: ", categorical_cols)

Numerical columns:  ['loan_id', 'no_of_dependents', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value', 'debt_to_income_ratio', 'total_assets_value', 'loan_to_asset_ratio']
Categorical columns:  ['education', 'self_employed', 'loan_status']


# Removing unwanted columns

In [11]:
dataset = dataset.drop(columns=['loan_id','no_of_dependents','residential_assets_value', 'commercial_assets_value', 'bank_asset_value','luxury_assets_value'],axis=1)
dataset

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,loan_status,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
0,Graduate,No,9600000,29900000,12,778,Approved,3.114583,50700000,0.589744
1,Not Graduate,Yes,4100000,12200000,8,417,Rejected,2.975610,17000000,0.717647
2,Graduate,No,9100000,29700000,20,506,Rejected,3.263736,57700000,0.514731
3,Graduate,No,8200000,30700000,8,467,Rejected,3.743902,52700000,0.582543
4,Not Graduate,Yes,9800000,24200000,20,382,Rejected,2.469388,55000000,0.440000
...,...,...,...,...,...,...,...,...,...,...
4264,Graduate,Yes,1000000,2300000,12,317,Rejected,2.300000,7400000,0.310811
4265,Not Graduate,Yes,3300000,11300000,20,559,Approved,3.424242,20000000,0.565000
4266,Not Graduate,No,6500000,23900000,18,457,Rejected,3.676923,39000000,0.612821
4267,Not Graduate,No,4100000,12800000,8,780,Approved,3.121951,28800000,0.444444


# Checking for duplicates

In [12]:
duplicates = dataset[dataset.duplicated()]
duplicates

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,loan_status,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio


In [13]:
pd.get_dummies(dataset)

,income_annum,loan_amount,loan_term,cibil_score,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio,education_ Graduate,education_ Not Graduate,self_employed_ No,self_employed_ Yes,loan_status_ Approved,loan_status_ Rejected
0,9600000,29900000,12,778,3.114583,50700000,0.589744,True,False,True,False,True,False
1,4100000,12200000,8,417,2.975610,17000000,0.717647,False,True,False,True,False,True
2,9100000,29700000,20,506,3.263736,57700000,0.514731,True,False,True,False,False,True
3,8200000,30700000,8,467,3.743902,52700000,0.582543,True,False,True,False,False,True
4,9800000,24200000,20,382,2.469388,55000000,0.440000,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,1000000,2300000,12,317,2.300000,7400000,0.310811,True,False,False,True,False,True
4265,3300000,11300000,20,559,3.424242,20000000,0.565000,False,True,False,True,True,False
4266,6500000,23900000,18,457,3.676923,39000000,0.612821,False,True,True,False,False,True
4267,4100000,12800000,8,780,3.121951,28800000,0.444444,False,True,True,False,True,False


In [14]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
dataset['education'] = label_encoder.fit_transform(dataset['education'].str.strip())

In [15]:
dataset['self_employed'] = label_encoder.fit_transform(dataset['self_employed'].str.strip())

In [16]:
dataset['loan_status'] = label_encoder.fit_transform(dataset['loan_status'].str.strip())

In [17]:
dataset

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,loan_status,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
0,0,0,9600000,29900000,12,778,0,3.114583,50700000,0.589744
1,1,1,4100000,12200000,8,417,1,2.975610,17000000,0.717647
2,0,0,9100000,29700000,20,506,1,3.263736,57700000,0.514731
3,0,0,8200000,30700000,8,467,1,3.743902,52700000,0.582543
4,1,1,9800000,24200000,20,382,1,2.469388,55000000,0.440000
...,...,...,...,...,...,...,...,...,...,...
4264,0,1,1000000,2300000,12,317,1,2.300000,7400000,0.310811
4265,1,1,3300000,11300000,20,559,0,3.424242,20000000,0.565000
4266,1,0,6500000,23900000,18,457,1,3.676923,39000000,0.612821
4267,1,0,4100000,12800000,8,780,0,3.121951,28800000,0.444444


In [18]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   education             4269 non-null   int64  
 1   self_employed         4269 non-null   int64  
 2   income_annum          4269 non-null   int64  
 3   loan_amount           4269 non-null   int64  
 4   loan_term             4269 non-null   int64  
 5   cibil_score           4269 non-null   int64  
 6   loan_status           4269 non-null   int64  
 7   debt_to_income_ratio  4269 non-null   float64
 8   total_assets_value    4269 non-null   int64  
 9   loan_to_asset_ratio   4269 non-null   float64
dtypes: float64(2), int64(8)
memory usage: 333.6 KB


In [19]:
dataset.columns

Index(['education', 'self_employed', 'income_annum', 'loan_amount',
       'loan_term', 'cibil_score', 'loan_status', 'debt_to_income_ratio',
       'total_assets_value', 'loan_to_asset_ratio'],
      dtype='object')

In [20]:
independent=dataset[['education', 'self_employed', 'income_annum', 'loan_amount',
       'loan_term', 'cibil_score',  'debt_to_income_ratio',
       'total_assets_value', 'loan_to_asset_ratio'
       ]]
dependent=dataset['loan_status']

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.3,random_state=0)

In [22]:
X_train

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
1023,1,1,1100000,2900000,4,554,2.636364,9500000,0.305263
728,1,0,3400000,8600000,16,312,2.529412,23900000,0.359833
133,1,1,6800000,13900000,8,479,2.044118,56100000,0.247772
2255,1,0,8900000,24000000,6,380,2.696629,59100000,0.406091
1044,0,1,2400000,4800000,12,895,2.000000,17700000,0.271186
...,...,...,...,...,...,...,...,...,...
1033,0,0,600000,1200000,2,752,2.000000,4600000,0.260870
3264,1,1,800000,2100000,12,363,2.625000,6200000,0.338710
1653,1,1,6400000,23500000,12,684,3.671875,60200000,0.390365
2607,1,0,5400000,12600000,18,786,2.333333,30400000,0.414474


In [23]:
X_test

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
1972,0,1,8300000,17700000,10,673,2.132530,61900000,0.285945
528,0,1,7200000,23900000,4,559,3.319444,45000000,0.531111
3540,0,0,7900000,15800000,8,551,2.000000,31300000,0.504792
87,1,1,8700000,28300000,8,402,3.252874,72100000,0.392510
1621,1,1,7200000,15600000,2,362,2.166667,31800000,0.490566
...,...,...,...,...,...,...,...,...,...
3559,1,0,2700000,10700000,6,857,3.962963,22600000,0.473451
4149,1,0,700000,2800000,2,561,4.000000,4000000,0.700000
3387,0,1,300000,800000,18,774,2.666667,1900000,0.421053
3649,1,1,2300000,7700000,16,589,3.347826,19000000,0.405263


In [24]:
y_train

1023    0
728     1
133     1
2255    1
1044    0
       ..
1033    0
3264    1
1653    0
2607    0
2732    0
Name: loan_status, Length: 2988, dtype: int64

In [25]:
y_test

1972    0
528     0
3540    0
87      1
1621    1
       ..
3559    0
4149    0
3387    0
3649    0
2355    1
Name: loan_status, Length: 1281, dtype: int64

# Standard scaler

In [26]:
from sklearn.preprocessing import StandardScaler
# Initialize StandardScaler
sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [27]:
dataset.describe()

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,loan_status,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
count,4269.000000,4269.000000,4.269000e+03,4.269000e+03,4269.000000,4269.000000,4269.000000,4269.000000,4.269000e+03,4269.000000
mean,0.497775,0.503631,5.059124e+06,1.513345e+07,10.900445,599.936051,0.377840,2.984807,3.255008e+07,0.486470
std,0.500054,0.500045,2.806840e+06,9.043363e+06,5.709187,172.430401,0.484904,0.595496,1.950533e+07,0.144564
min,0.000000,0.000000,2.000000e+05,3.000000e+05,2.000000,300.000000,0.000000,1.500000,5.000000e+05,0.176471
25%,0.000000,0.000000,2.700000e+06,7.700000e+06,6.000000,453.000000,0.000000,2.464286,1.630000e+07,0.382022
50%,0.000000,1.000000,5.100000e+06,1.450000e+07,10.000000,600.000000,0.000000,3.000000,3.150000e+07,0.466667
75%,1.000000,1.000000,7.500000e+06,2.150000e+07,16.000000,748.000000,1.000000,3.500000,4.720000e+07,0.565693
max,1.000000,1.000000,9.900000e+06,3.950000e+07,20.000000,900.000000,1.000000,4.000000,9.070000e+07,1.333333


# Calculating and removing outliers

In [28]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile) for 'person_income'
Q1 = dataset['total_assets_value'].quantile(0.25)
Q3 = dataset['total_assets_value'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(lower_bound)
print(upper_bound)

-30050000.0
93550000.0


In [29]:
dataset = dataset[(dataset['total_assets_value'] >= lower_bound) & (dataset['total_assets_value'] <= upper_bound)]
dataset

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,loan_status,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
0,0,0,9600000,29900000,12,778,0,3.114583,50700000,0.589744
1,1,1,4100000,12200000,8,417,1,2.975610,17000000,0.717647
2,0,0,9100000,29700000,20,506,1,3.263736,57700000,0.514731
3,0,0,8200000,30700000,8,467,1,3.743902,52700000,0.582543
4,1,1,9800000,24200000,20,382,1,2.469388,55000000,0.440000
...,...,...,...,...,...,...,...,...,...,...
4264,0,1,1000000,2300000,12,317,1,2.300000,7400000,0.310811
4265,1,1,3300000,11300000,20,559,0,3.424242,20000000,0.565000
4266,1,0,6500000,23900000,18,457,1,3.676923,39000000,0.612821
4267,1,0,4100000,12800000,8,780,0,3.121951,28800000,0.444444


In [30]:
dataset= dataset[dataset['cibil_score'] <1000]

In [31]:
dataset

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,loan_status,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
0,0,0,9600000,29900000,12,778,0,3.114583,50700000,0.589744
1,1,1,4100000,12200000,8,417,1,2.975610,17000000,0.717647
2,0,0,9100000,29700000,20,506,1,3.263736,57700000,0.514731
3,0,0,8200000,30700000,8,467,1,3.743902,52700000,0.582543
4,1,1,9800000,24200000,20,382,1,2.469388,55000000,0.440000
...,...,...,...,...,...,...,...,...,...,...
4264,0,1,1000000,2300000,12,317,1,2.300000,7400000,0.310811
4265,1,1,3300000,11300000,20,559,0,3.424242,20000000,0.565000
4266,1,0,6500000,23900000,18,457,1,3.676923,39000000,0.612821
4267,1,0,4100000,12800000,8,780,0,3.121951,28800000,0.444444


In [32]:
dataset.describe()

,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,loan_status,debt_to_income_ratio,total_assets_value,loan_to_asset_ratio
count,4269.000000,4269.000000,4.269000e+03,4.269000e+03,4269.000000,4269.000000,4269.000000,4269.000000,4.269000e+03,4269.000000
mean,0.497775,0.503631,5.059124e+06,1.513345e+07,10.900445,599.936051,0.377840,2.984807,3.255008e+07,0.486470
std,0.500054,0.500045,2.806840e+06,9.043363e+06,5.709187,172.430401,0.484904,0.595496,1.950533e+07,0.144564
min,0.000000,0.000000,2.000000e+05,3.000000e+05,2.000000,300.000000,0.000000,1.500000,5.000000e+05,0.176471
25%,0.000000,0.000000,2.700000e+06,7.700000e+06,6.000000,453.000000,0.000000,2.464286,1.630000e+07,0.382022
50%,0.000000,1.000000,5.100000e+06,1.450000e+07,10.000000,600.000000,0.000000,3.000000,3.150000e+07,0.466667
75%,1.000000,1.000000,7.500000e+06,2.150000e+07,16.000000,748.000000,1.000000,3.500000,4.720000e+07,0.565693
max,1.000000,1.000000,9.900000e+06,3.950000e+07,20.000000,900.000000,1.000000,4.000000,9.070000e+07,1.333333
